# Backfill Price Gaps

Scans all `data/<year>/PRICES_*.csv` for **tickers you specify**, detects missing trading days, and backfills from the same data source.

Trading days are derived from the data itself (dates where any ticker has a row), so **holidays and weekends are never treated as gaps**. Per-ticker ranges are bounded by each ticker's first appearance, so **pre-IPO dates are never flagged**.

In [1]:
import sys
from datetime import date, timedelta
from pathlib import Path

import pandas as pd

_root = Path.cwd().resolve()
while _root != _root.parent and not (_root / ".git").exists():
    _root = _root.parent
sys.path.insert(0, str(_root))

from research.functions.download_helper import (
    find_project_root,
    normalize_dates,
    split_into_contiguous_ranges,
)
from research.config.constants import get_universe
from research.functions.fetch_and_store import fetch_and_store

PROJECT_ROOT = find_project_root(Path.cwd())

In [2]:
DATA_DIR = PROJECT_ROOT / "data"
END_DATE = date.today()

TICKERS_TO_BACKFILL = get_universe()

## 1. Load all existing data (vectorised)

Build two things from the CSVs:
- **`trading_dates`** — set of dates where *any* ticker has a row (= actual market open days).
- **`ticker_dates`** — `{ticker: set of dates}` for the tickers we care about.

In [3]:
all_dates: set[date] = set()
ticker_dates: dict[str, set[date]] = {t: set() for t in TICKERS_TO_BACKFILL}
ticker_set = set(TICKERS_TO_BACKFILL)

for path in sorted(DATA_DIR.rglob("PRICES_*.csv")):
    try:
        df = pd.read_csv(path, usecols=["date", "ticker"], parse_dates=["date"])
        df = normalize_dates(df)
        # Collect all trading dates
        dates_in_file = set(df["date"].unique())
        # Convert numpy dates to python dates
        dates_in_file = {d.date() if hasattr(d, "date") else d for d in dates_in_file}
        all_dates.update(dates_in_file)
        # Collect per-ticker dates
        for t in ticker_set & set(df["ticker"].unique()):
            t_dates = set(df.loc[df["ticker"] == t, "date"].unique())
            t_dates = {d.date() if hasattr(d, "date") else d for d in t_dates}
            ticker_dates[t].update(t_dates)
    except Exception as e:
        print(f"Skip {path.name}: {e}")

trading_dates = sorted(all_dates)
print(f"Trading dates in data: {len(trading_dates)} ({trading_dates[0]} → {trading_dates[-1]})")
for t in TICKERS_TO_BACKFILL:
    print(f"  {t}: {len(ticker_dates[t])} dates")

Trading dates in data: 1495 (2020-01-02 → 2026-02-09)
  SPY: 1420 dates
  IVV: 1451 dates
  VOO: 1455 dates
  SPLG: 1450 dates
  RSP: 1420 dates
  VTI: 1451 dates
  ITOT: 1433 dates
  SCHB: 1279 dates
  IWV: 1279 dates
  IWD: 1483 dates
  IWF: 1300 dates
  SCHX: 1279 dates
  VV: 1223 dates
  VTV: 1279 dates
  VUG: 1279 dates
  MDY: 1300 dates
  IJH: 1299 dates
  IWR: 1260 dates
  IWM: 1279 dates
  VB: 1260 dates
  SCHA: 1279 dates
  VXF: 1139 dates
  DIA: 1141 dates
  QQQ: 1279 dates
  QQQM: 1089 dates
  XLK: 1299 dates
  VGT: 1279 dates
  IYW: 1319 dates
  IGV: 1279 dates
  SMH: 1279 dates
  SOXX: 1279 dates
  XLV: 1340 dates
  VHT: 1299 dates
  IYH: 1300 dates
  XBI: 1320 dates
  XLF: 1260 dates
  VFH: 1319 dates
  IYF: 1300 dates
  KBE: 1483 dates
  KRE: 1483 dates
  XLE: 1279 dates
  VDE: 1141 dates
  IYE: 1279 dates
  XLI: 1483 dates
  VIS: 1121 dates
  IYJ: 1300 dates
  PAVE: 1299 dates
  XLY: 1260 dates
  VCR: 1483 dates
  IYC: 1483 dates
  XHB: 1279 dates
  XLP: 1280 dates
  VD

## 2. Find gaps per ticker

For each ticker, gaps = trading dates between the ticker's **first and last** existing date that it is missing. This avoids pre-IPO false positives and holiday false positives in one shot.

In [4]:
gaps_by_ticker: dict[str, list[date]] = {}
trading_dates_set = set(trading_dates)

for t in TICKERS_TO_BACKFILL:
    if not ticker_dates[t]:
        print(f"  {t}: no existing data, skipping")
        continue
    first = min(ticker_dates[t])
    last = max(ticker_dates[t])
    # Expected = trading dates in [first, last] for this ticker
    expected = {d for d in trading_dates_set if first <= d <= last}
    missing = sorted(expected - ticker_dates[t])
    if missing:
        gaps_by_ticker[t] = missing

total_gaps = sum(len(v) for v in gaps_by_ticker.values())
print(f"Total gaps: {total_gaps} across {len(gaps_by_ticker)} tickers")
for t, dates in gaps_by_ticker.items():
    print(f"  {t}: {len(dates)} missing days ({dates[0]} → {dates[-1]})")

  IEU: no existing data, skipping
Total gaps: 63391 across 314 tickers
  SPY: 75 missing days (2020-11-06 → 2022-05-27)
  IVV: 44 missing days (2020-12-04 → 2022-03-31)
  VOO: 40 missing days (2020-12-04 → 2022-03-31)
  SPLG: 45 missing days (2020-12-04 → 2025-10-24)
  RSP: 75 missing days (2020-11-06 → 2022-05-27)
  VTI: 44 missing days (2020-12-04 → 2022-03-31)
  ITOT: 62 missing days (2020-11-30 → 2025-12-31)
  SCHB: 216 missing days (2020-11-30 → 2025-12-31)
  IWV: 216 missing days (2020-11-30 → 2025-12-31)
  IWD: 12 missing days (2020-11-30 → 2025-12-31)
  IWF: 195 missing days (2020-11-30 → 2025-12-31)
  SCHX: 216 missing days (2020-11-30 → 2025-12-31)
  VV: 272 missing days (2020-11-02 → 2025-12-31)
  VTV: 216 missing days (2020-11-30 → 2025-12-31)
  VUG: 216 missing days (2020-11-30 → 2025-12-31)
  MDY: 195 missing days (2020-11-30 → 2025-12-31)
  IJH: 196 missing days (2020-11-30 → 2025-12-31)
  IWR: 235 missing days (2020-11-02 → 2025-12-31)
  IWM: 216 missing days (2020-11-3

## 3. Fetch and merge into monthly CSVs

Gap dates are split into **tight contiguous ranges** (new range when consecutive gaps are >30 days apart), then tickers sharing the same date range are **batched into a single yfinance call** (up to 20 tickers per batch). Only gap dates are kept before merging.

In [5]:
ticker_ranges = {
    t: split_into_contiguous_ranges(gaps, max_gap_days=30)
    for t, gaps in gaps_by_ticker.items()
}
filter_dates = {t: set(gaps) for t, gaps in gaps_by_ticker.items()}

total_ranges = sum(len(r) for r in ticker_ranges.values())
print(f"Fetching {total_ranges} ranges across {len(ticker_ranges)} tickers (batched)")
for t, ranges in ticker_ranges.items():
    print(f"  {t}: {len(ranges)} range(s)")

result = fetch_and_store(
    ticker_ranges, DATA_DIR,
    filter_dates=filter_dates,
    on_ticker=lambda t, n: print(f"  {t}: {n} rows backfilled"),
)
print(f"\nDone. Total rows backfilled: {sum(result.stored.values())}")
if result.failed:
    print(f"WARNING: {len(result.failed)} tickers returned no data: {result.failed}")

Fetching 2080 ranges across 314 tickers (batched)
  SPY: 1 range(s)
  IVV: 4 range(s)
  VOO: 5 range(s)
  SPLG: 5 range(s)
  RSP: 1 range(s)
  VTI: 4 range(s)
  ITOT: 10 range(s)
  SCHB: 6 range(s)
  IWV: 6 range(s)
  IWD: 10 range(s)
  IWF: 7 range(s)
  SCHX: 6 range(s)
  VV: 8 range(s)
  VTV: 6 range(s)
  VUG: 6 range(s)
  MDY: 7 range(s)
  IJH: 6 range(s)
  IWR: 6 range(s)
  IWM: 6 range(s)
  VB: 6 range(s)
  SCHA: 6 range(s)
  VXF: 5 range(s)
  DIA: 5 range(s)
  QQQ: 6 range(s)
  QQQM: 6 range(s)
  XLK: 7 range(s)
  VGT: 6 range(s)
  IYW: 8 range(s)
  IGV: 6 range(s)
  SMH: 6 range(s)
  SOXX: 6 range(s)
  XLV: 9 range(s)
  VHT: 7 range(s)
  IYH: 7 range(s)
  XBI: 7 range(s)
  XLF: 6 range(s)
  VFH: 8 range(s)
  IYF: 7 range(s)
  KBE: 10 range(s)
  KRE: 10 range(s)
  XLE: 6 range(s)
  VDE: 5 range(s)
  IYE: 6 range(s)
  XLI: 10 range(s)
  VIS: 4 range(s)
  IYJ: 7 range(s)
  PAVE: 7 range(s)
  XLY: 6 range(s)
  VCR: 10 range(s)
  IYC: 10 range(s)
  XHB: 6 range(s)
  XLP: 7 range(s)
 

In [6]:
## Re-check gaps after backfill
all_dates_v: set[date] = set()
ticker_dates_v: dict[str, set[date]] = {t: set() for t in TICKERS_TO_BACKFILL}
ticker_set_v = set(TICKERS_TO_BACKFILL)

for path in sorted(DATA_DIR.rglob("PRICES_*.csv")):
    try:
        df = pd.read_csv(path, usecols=["date", "ticker"], parse_dates=["date"])
        df = normalize_dates(df)
        dates_in_file = {d.date() if hasattr(d, "date") else d for d in df["date"].unique()}
        all_dates_v.update(dates_in_file)
        for t in ticker_set_v & set(df["ticker"].unique()):
            t_dates = {d.date() if hasattr(d, "date") else d for d in df.loc[df["ticker"] == t, "date"].unique()}
            ticker_dates_v[t].update(t_dates)
    except Exception:
        continue

trading_dates_v = set(all_dates_v)
remaining_gaps = 0
for t in TICKERS_TO_BACKFILL:
    if not ticker_dates_v[t]:
        continue
    first = min(ticker_dates_v[t])
    last = max(ticker_dates_v[t])
    expected = {d for d in trading_dates_v if first <= d <= last}
    missing = expected - ticker_dates_v[t]
    if missing:
        remaining_gaps += len(missing)
        print(f"  {t}: {len(missing)} gaps remain")

print(f"\nRemaining gaps after backfill: {remaining_gaps}")
if remaining_gaps == 0:
    print("All gaps resolved.")
else:
    print(f"WARNING: {remaining_gaps} gaps remain. Check failed tickers above.")

  SPLG: 1 gaps remain

Remaining gaps after backfill: 1
